In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tools.config import PROCESSED_DATA_DIR, INTERIM_DATA_DIR
import logging
import numpy as np
from argparse import ArgumentParser

In [5]:
%tb
parser = ArgumentParser(add_help=False)
# Dataset parameters
parser.add_argument('--dataset-path', type=str, default=PROCESSED_DATA_DIR / 'smallerSampleDataset')
parser.add_argument('--reshape-size', default=[224, 224, 3])
parser.add_argument('--phishing-test-size', default=0.4)
parser.add_argument('--num-targets', type=int, default=5)
parser.add_argument('--legit-imgs-num', default=420)
parser.add_argument('--phish-imgs-num', default=160)
# Model parameters
parser.add_argument('--input-shape', default=[224, 224, 3])
parser.add_argument('--margin', type=float, default=2.2)
parser.add_argument('--new-conv-params', default=[5, 5, 512])
# Training parameters
parser.add_argument('--start-lr', type=float, default=2e-5)  # 0.00002
parser.add_argument('--output-dir', type=str, default=INTERIM_DATA_DIR / 'smallerSampleDataset')
parser.add_argument('--saved-model-name', type=str, default='model')  # from first training
parser.add_argument('--new-saved-model-name', type=str, default='model2')
parser.add_argument('--save-interval', type=int, default=2000)
parser.add_argument('--batch-size', type=int, default=32)
parser.add_argument('--n-iter', type=int, default=50000)
parser.add_argument('--lr-interval', type=int, default=250)
# hard examples training
parser.add_argument('--num-sets', type=int, default=100)
parser.add_argument('--iter-per-set', type=int, default=8)
# parser.add_argument('--n_iter', type=int, default=30)


logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s",
)
logger = logging.getLogger()

SystemExit: 2

In [6]:
args, _ = parser.parse_known_args()

In [7]:
# Define paths for saved .npy files
imgs_train_path = args.output_dir / 'all_imgs_train.npy'
labels_train_path = args.output_dir / 'all_labels_train.npy'
file_names_train_path = args.output_dir / 'all_file_names_train.npy'

imgs_test_path = args.output_dir / 'all_imgs_test.npy'
labels_test_path = args.output_dir / 'all_labels_test.npy'
file_names_test_path = args.output_dir / 'all_file_names_test.npy'

# Initialize variables
all_imgs_train, all_labels_train, all_file_names_train = None, None, None
all_imgs_test, all_labels_test, all_file_names_test = None, None, None
data_path_trusted = args.dataset_path / 'trusted_list'
data_path_phish = args.dataset_path / 'phishing'

# Check if all .npy files exist
if (imgs_train_path.exists() and labels_train_path.exists() and file_names_train_path.exists() and
        imgs_test_path.exists() and labels_test_path.exists() and file_names_test_path.exists()):
    logger.info('Loading pre-saved data')

    # Load pre-saved data
    all_imgs_train = np.load(imgs_train_path)
    all_labels_train = np.load(labels_train_path)
    all_file_names_train = np.load(file_names_train_path)

    all_imgs_test = np.load(imgs_test_path)
    all_labels_test = np.load(labels_test_path)
    all_file_names_test = np.load(file_names_test_path)

2025-01-19 23:57:47,985 INFO Loading pre-saved data


In [8]:
logger.info('Images loaded')

X_train_legit = all_imgs_train
y_train_legit = all_labels_train
# Load the same train/split in phase 1
phish_test_idx = np.load(args.output_dir / 'test_idx.npy')
phish_train_idx = np.load(args.output_dir / 'train_idx.npy')

X_test_phish = all_imgs_test[phish_test_idx, :]
y_test_phish = all_labels_test[phish_test_idx, :]

X_train_phish = all_imgs_test[phish_train_idx, :]
y_train_phish = all_labels_test[phish_train_idx, :]

2025-01-19 23:57:50,228 INFO Images loaded


In [34]:
from trainer_phase2 import all_targets_start_end, HardSubsetSampling, prepare_model, start_end_each_target
from triplet_sampling import TargetHelper, get_batch

In [18]:
labels_start_end_train_legit = all_targets_start_end(args.num_targets, y_train_legit)
labels_start_end_train_legit

array([[  0.,  69.],
       [ 70., 217.],
       [218., 262.],
       [263., 365.],
       [366., 419.]])

In [17]:
hard_subset_sampling = HardSubsetSampling()

In [19]:
hard_subset_sampling.find_fixed_set_idx(
            labels_start_end_train_legit=labels_start_end_train_legit, num_target=args.num_targets)

array([ 69., 212., 241., 275., 376.])

In [27]:
targetHelper = TargetHelper(data_path_phish)
full_model = prepare_model(args)

NameError: Exception encountered when calling layer "lambda_1" (type Lambda).

name 'K' is not defined

Call arguments received by layer "lambda_1" (type Lambda):
  • inputs=['tf.Tensor(shape=(None, 512), dtype=float32)', 'tf.Tensor(shape=(None, 512), dtype=float32)']
  • mask=None
  • training=None

In [ ]:
fixed_set_idx = hard_subset_sampling.find_fixed_set_idx(
            labels_start_end_train_legit=labels_start_end_train_legit, num_target=args.num_targets)
fixed_set = X_train_legit[fixed_set_idx.astype(int), :, :, :]
fixed_set

In [35]:
n=1
X_train = np.concatenate([X_train_legit, X_train_phish])
y_train = np.concatenate([y_train_legit, y_train_phish])

X_train_new = np.zeros(
        [args.num_targets * 2 * n, X_train_legit.shape[1], X_train_legit.shape[2], X_train_legit.shape[3]])
y_train_new = np.zeros([args.num_targets * 2 * n, 1])



model = full_model.layers[3]
X_train_new, y_train_new, labels_start_end_train = hard_subset_sampling.find_main_train(
    model=model,
    fixed_set=fixed_set,
    targets=args.num_targets,
    X_train=X_train,
    y_train=y_train,
    X_train_new=X_train_new,
    y_train_new=y_train_new,
    X_train_legit=X_train_legit,
    X_train_phish=X_train_phish
)
labels_start_end_train

NameError: name 'full_model' is not defined

In [36]:
get_batch(
    targetHelper=targetHelper,
    X_train_legit=X_train_legit,
    X_train_new=X_train_new,
    labels_start_end_train=labels_start_end_train,
    batch_size=args.batch_size,
    train_fixed_set=fixed_set,
    num_targets=args.num_targets
)

NameError: name 'labels_start_end_train' is not defined